# Parallel embedding

<span style="font-size: 16pt; font-style: italic;">Study</span>

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)    
September 2024  

In [ ]:
# use Data::Generators;
# use Data::Importers;
# use Data::Summarizers;
# use Data::Reshapers;

use WWW::LLaMA::Embeddings;

In [ ]:
my @content = <angel apple ardvark bible car cat cherry chocolate cookie cow devil film horse house movie plum projector raccoon tiger tree>;
#my @content = random-word(1000);
@content.elems

In [ ]:
llama-model

In [ ]:
'What is the distance from Earth to Mars? In kilometers.' ~ llm-prompt('NumericOnly')
    

In [ ]:
llama-chat-completion(
    'What is the distance from Earth to Mars? In kilometers.' ~ llm-prompt('NumericOnly'),
    temperature => 0.4, 
    format => 'JSON')

In [ ]:
my $format = 'values';
my $method = 'tiny';
my $conf = llm-configuration(
    'LLaMA', 
    model => 'llama-embedding',
    base-url => 'http://127.0.0.1:8080',
    embedding-model => 'llama-embedding',
    embedding-function => &WWW::LLaMA::Embeddings::LLaMAEmbeddings,
    module => 'WWW::LLaMA'
    );
$conf.Hash.elems

In [ ]:
my $tstart = now;
my @vector-embeddings = llama-embedding(@content, :$format, :$method, :!batched):!echo;
my $tend = now;

say "Sequential embeddings computation timings:";
say "total   : {$tend - $tstart} seconds.";
say "average : {($tend - $tstart)/@content.elems} seconds.";

In [ ]:
45_000 * 0.0394740421

In [ ]:
say @vector-embeddings.elems;
sink @vector-embeddings».elems ==> records-summary()

In [ ]:
@vector-embeddings[10]

In [ ]:
my $tstart = now;

my $degree = 2;
my $batch = round(@content.elems / $degree);

say (:$degree, :$batch);

#my @vector-embeddings = @content.rotor($batch, :partial).race(:$degree, :$batch).map({ note $_; llm-embedding($_, :$llm-evaluator) }).flat;
my @vector-embeddings = @content.rotor($batch, :partial).race(:$degree, :$batch).map({ note $_; llama-embedding($_, :$format, :$method) }).map(*.Slip);

say "Number of embeddings : {@vector-embeddings.elems}";
say "Embedding length     : {@vector-embeddings.head.elems}";

my $tend = now;

say "Time for parrallel embeddings computation: {$tend - $tstart} seconds."

In [ ]:
@vector-embeddings.head

In [ ]:
deduce-type(@vector-embeddings):tally

In [ ]:
say @vector-embeddings.elems;
sink @vector-embeddings».elems ==> records-summary()